In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchsummary import summary
import wandb

from urnn import URNN

In [ ]:
hyperparameters = {
    'epochs': 20,
    'batch_size': 1,
    'learning_rate': 1e-4,
}

wandb.init(
    project='regression-neural-network',
    name='test run 3',
    entity='konradszafer'
)
wandb.config = hyperparameters

In [ ]:
data = np.linspace(0, 1.0, 30)
# data += np.random.normal(0, .02, data.shape)
plt.plot(data)
plt.show()

In [ ]:
x, y = [], []
window = 3
for i in range(0, len(data)-window):
    x.append(data[i:i+window])
    y.append(data[i+window])
x, y = np.array(x), np.array(y)
x, y = Tensor(x), Tensor(y)

In [ ]:
model = URNN(
    input_size=window,
    min_value=0.0,
    max_value=1+1e-2,
    latent_resolution=5
)

y = model.digitize(y)
y

In [ ]:
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=hyperparameters['batch_size'], shuffle=True)
dataset[0]

In [ ]:
epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


for epoch in range(1, epochs+1):
    for seq, target in dataloader:

        output = model(seq)

        print(seq, target)
        print(output)

        # loss = model.loss(output, target)
        loss = F.cross_entropy(output, target, reduction='mean')
        loss = loss.sum() / hyperparameters['batch_size']
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        _, output = torch.max(output, 1)
        # output.eq(target).sum().item()
        # print(output)

        print(loss)
        wandb.log({'loss': loss})
